<a href="https://colab.research.google.com/github/pratikmehkarkar/TfLiteModelTester/blob/main/TfliteTester.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf

# Load the TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path="model.tflite")
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
input_details

In [ ]:
output_details = interpreter.get_output_details()
output_details

In [ ]:
from PIL import Image

# im = Image.open("animal.jpg")
im = Image.open("1357.jpeg")
# im = Image.open("banana.jpg")

In [ ]:
print(im.size)

In [ ]:
res_im = im.resize((300, 300))
res_im

In [ ]:
np_res_im = np.array(res_im)
np_res_im = (np_res_im).astype('float32')
np_res_im

In [ ]:
input_details[0]['shape']

In [ ]:
np_res_im.shape

In [ ]:
if len(np_res_im.shape) == 3:
    np_res_im = np.expand_dims(np_res_im, 0)
# Test the model on random input data.
input_shape = input_details[0]['shape']
input_data = np_res_im
interpreter.set_tensor(input_details[0]['index'], input_data)

interpreter.invoke()

# The function `get_tensor()` returns a copy of the tensor data.
# Use `tensor()` in order to get a pointer to the tensor.
output_data = interpreter.get_tensor(output_details[0]['index'])

In [ ]:
output_data.shape

In [ ]:
output_data[0].shape

In [ ]:
classification_prob = []
classification_label = []
total = 0
for index,prob in enumerate(output_data[0]):
    if prob != 0:
        classification_prob.append(prob)
        total += prob
        classification_label.append(index)

In [ ]:
label_names = [line.rstrip('\n') for line in open("mylabel.txt")]
label_names

In [ ]:
found_labels = np.array(label_names)[classification_label]

In [ ]:
import pandas as pd

df = pd.DataFrame(classification_prob/total, found_labels)
sorted_df = df.sort_values(by=0,ascending=False)
sorted_df